In [1]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "airesearch/wangchanberta-base-att-spm-uncased"
batch_size = 12
label_list = ['O', 'B-c', 'I-c', 'B-p', 'I-p']

## Get data

In [2]:
import dill
import pandas as pd
from sklearn.model_selection import train_test_split

In [3]:
path_name = "../dataset/data/"

with open(path_name + 'comment-pos.data', 'rb') as file:
    datatofile = dill.load(file)

tagged_sents = []
for data in datatofile:
    text_inside = []
    for word, pos, label in data:
        if word.strip() == '':
            text_inside.append((' ', pos, label))
        else:
            text_inside.append((word, pos, label))
    tagged_sents.append(text_inside)

train_sents, test_sents = train_test_split(tagged_sents, test_size=0.2, random_state=42)
print(len(train_sents))
print(len(test_sents))
print(train_sents[1])

552
138
[('ถ้า', 'JSBR', 'B-c'), ('เดินทาง', 'VACT', 'I-c'), ('กลางคืน', 'NCMN', 'I-c'), ('ก็', 'JSBR', 'I-c'), ('รถทัวร์', 'NCMN', 'I-c'), ('ครับ', 'NCMN', 'I-c'), (' ', 'PUNC', 'I-c'), (' ', 'PUNC', 'O'), ('เพราะ', 'JSBR', 'B-p'), ('รถ', 'NCMN', 'I-p'), ('ไม่', 'NEG', 'I-p'), ('เยอะ', 'VSTA', 'I-p'), (' ', 'PUNC', 'I-p'), ('ความเสี่ยง', 'NCMN', 'I-p'), ('การ', 'FIXN', 'I-p'), ('เกิด', 'VSTA', 'I-p'), ('อุบัติ', 'NCMN', 'I-p'), ('ห', 'NCMN', 'I-p'), ('ตุ', 'NCMN', 'I-p'), ('ก็', 'JSBR', 'I-p'), ('น้อย', 'VATT', 'I-p'), ('(', 'PUNC', 'I-p'), ('มั้ง', 'JCRG', 'I-p'), (')', 'PUNC', 'I-p'), (' ', 'PUNC', 'I-p'), (' ', 'PUNC', 'O'), ('ถ้า', 'JSBR', 'B-c'), ('กลางวัน', 'NCMN', 'I-c'), ('ก็', 'JSBR', 'I-c'), ('เครื่องบิน', 'VSTA', 'I-c'), ('ครับ', 'NCMN', 'I-c'), (' ', 'PUNC', 'I-c'), (' ', 'PUNC', 'O'), ('เพราะ', 'JSBR', 'B-p'), (' ', 'PUNC', 'I-p'), ('มัน', 'PPRS', 'I-p'), ('ใช้เวลา', 'VSTA', 'I-p'), ('น้อย', 'ADVN', 'I-p'), ('จะ', 'XVBM', 'I-p'), ('ได้', 'XVAM', 'I-p'), ('มี', 'VSTA', 'I-

In [4]:
def data_to_dict(data):
    datasets = dict()
    datasets['id'] = []
    datasets['tokens'] = []
    datasets['pos_tags'] = []
    datasets['ner_tags'] = []
    for i, sent in enumerate(data):
        _word = []
        _label = []
        _pos = []
        for word, pos, label in sent:
            _word.append(word)
            _label.append(label)
            _pos.append(pos)
        datasets['id'].append(i)
        datasets['tokens'].append(_word)
        datasets['pos_tags'].append(_pos)
        datasets['ner_tags'].append([label_list.index(l) for l in _label])
    return datasets

In [5]:
from datasets import DatasetDict, Dataset,Sequence, Features, Value, ClassLabel

ft = Features({
    'id': Value("int32"),
    'tokens': Sequence(Value("string")), 
    'pos_tags': Sequence(Value("string")),
    'ner_tags': Sequence(ClassLabel(names=label_list))
    })

datasets = DatasetDict({
    'train': Dataset.from_dict(data_to_dict(train_sents), features=ft),
    'test': Dataset.from_dict(data_to_dict(test_sents), features=ft)
})

datasets

C:\Users\pitiw\miniconda3\envs\wangchan\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 552
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags'],
        num_rows: 138
    })
})

In [6]:
label_list = datasets["train"].features[f"{task}_tags"].feature.names
print('len:', len(label_list))
print(label_list)

len: 5
['O', 'B-c', 'I-c', 'B-p', 'I-p']


In [7]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

show_random_elements(datasets["train"])

,id,tokens,pos_tags,ner_tags
0,291,"[โดยส่วนตัว, แล้ว, อยาก, ให้, สถานศึกษา, เป็น, คน, กำหนด, มากกว่า, , เพราะว่า, ถ้า, คุณ, ต้องการ, จะ, ศึกษา, ที่, โรงเรียน, ไหน, ๆ, , คุณ, ควรจะ, ยอมรับ, กฎเกณฑ์, ของ, แต่ละ, โรงเรียน, นั้น, ได้, , ใช่, ค่ะ, , เครื่องแบบ, ไม่, ได้, เป็นความ, บาง, ชี้, ถึง, ผลการเรียน, , อย่างไรก็ตาม, คุณ, ต้อง, ยอมรับ, กฎ, ของ, สถานศึกษา, ถ้า, คุณ, อยาก, เรียน, ก็, ต้อง, ทำตาม, กฎ, ของ, โรงเรียน, ค่ะ]","[NCMN, XVAE, XVAM, VACT, NCMN, VSTA, NCMN, VACT, ADVN, PUNC, JSBR, JSBR, PPRS, VACT, XVBM, VACT, PREL, NCMN, PNTR, NCMN, PUNC, PPRS, XVMM, VACT, NCMN, RPRE, DIBQ, NCMN, DDAC, XVAE, PUNC, VSTA, NCMN, PUNC, NCMN, NEG, XVAE, VACT, DIBQ, VACT, RPRE, NCMN, PUNC, JSBR, VACT, XVMM, VACT, NCMN, RPRE, NCMN, JSBR, VACT, NPRP, VACT, JSBR, XVMM, VACT, NCMN, RPRE, NCMN, NCMN]","[B-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
1,350,"[เกมส์, เป็น, แค่, สื่อ, ๆ, หนึ่ง, , ไม่, ต่าง, จาก, , หนัง, , ละคร, , รายการโทรทัศน์, , เพลง, , หรือ, มหสพ, ครับ, , ใช้, ให้, ถูก, ทาง, คนใน, , Silicon, , Valley, , ที่, เห็น, เป็น, ผู้ชาย, เยอะ, ๆ, เพราะ, เล่น, เกมส์, มา, ตั้งแต่, เด็ก, ๆ, ทั้งนั้น, ครับ, , เด็กผู้หญิง, ใน, ยุค, 80, -, 90, เล่น, VDO, เกมส์, น้อยกว่า, มาก, , มัน, เป็น, จุดเริ่มต้น, ของ, ความสนใจ, ใน, เรื่อง, เกี่ยวกับ, , IT, ครับ]","[NCMN, VSTA, NCMN, NCMN, NCMN, DCNM, PUNC, NEG, VSTA, RPRE, PUNC, NCMN, PUNC, NCMN, PUNC, NCMN, PUNC, NCMN, PUNC, JCRG, NCMN, NCMN, PUNC, VACT, JSBR, VATT, NCMN, NCMN, PUNC, NCMN, PUNC, NCMN, PUNC, PREL, VSTA, VSTA, NCMN, NCMN, PUNC, JSBR, VACT, NCMN, XVAE, RPRE, NCMN, NCMN, DDAN, DCNM, PUNC, VACT, RPRE, NCMN, NCNM, PUNC, DCNM, VACT, NCMN, VSTA, ADVN, ADVN, NCMN, PPRS, VSTA, NCMN, RPRE, NCMN, RPRE, NCMN, RPRE, PUNC, NCMN, NCMN]","[B-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
2,382,"[อ่าน, , คห, , 3, , เขียน, ได้ดี, , รู้, ลึก, , รู้, จริง, , ถ้า, จะ, ให้, ลูก, เรียน, , ปวช., ใน, กรุงเทพ, , ปริมณฑล, , อย่า, เรียน, , โรงเรียน, ที่, ชอบ, ตี, กัน, , และ, เป็นข่าว, , มี, โจทย์, เยอะ, จะ, หา, ความสบายใจ, ไม่, ได้, เลย, , ไม่, รู้, ว่า, ลูก, จะ, กลับ, ถึง, บ้าน, หรือเปล่า, , หรือ, จะ, นอน, อยู่, ข้าง, ถนน, , นี่, คือ, ข้อเสีย, ของ, การเรียน, สาย, อาชีพ, ในประเทศ, ไทย, , ทำให้, ไม่, อยาก, ให้, ลูก, เรียน]","[VACT, PUNC, NCMN, PUNC, DCNM, PUNC, VACT, NCMN, PUNC, VSTA, VATT, PUNC, VSTA, ADVN, PUNC, JSBR, XVBM, VACT, NCMN, VACT, PUNC, NCMN, RPRE, NCMN, PUNC, NCMN, PUNC, NCMN, VACT, PUNC, NCMN, PREL, VACT, VACT, ADVN, PUNC, JCRG, NCMN, PUNC, VSTA, NCMN, NCMN, XVBM, VACT, NCMN, NEG, XVAE, ADVN, PUNC, NEG, VSTA, JSBR, NCMN, XVBM, VACT, RPRE, NCMN, NCMN, PUNC, JCRG, XVBM, VSTA, XVAE, NCMN, NCMN, PUNC, PDMN, VSTA, NCMN, RPRE, NCMN, NCMN, NCMN, NCMN, NPRP, PUNC, VACT, NEG, XVMM, VACT, NCMN, VACT]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
3,458,"[เลือก, เงิน, ค่ะ, , , เพราะ, มี, , 20, , ล้าน, แล้ว, เดี๋ยว, หา, หนุ่ม, ๆ, , มา, ดูแล, ได้, ไม่, ยาก, ค่ะ, ]","[VACT, NCMN, NCMN, PUNC, PUNC, JSBR, VSTA, PUNC, DCNM, PUNC, CNIT, XVAE, VACT, VACT, NCMN, NCMN, PUNC, XVAE, VACT, XVAE, NEG, VATT, NCMN, PUNC]","[B-c, I-c, I-c, I-c, O, B-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p, I-p]"
4,381,

## Preprocess The data

In [8]:
from transformers import AutoTokenizer  
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, model_max_length=126)

In [9]:
import transformers
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)

In [10]:
example = datasets["train"][2]
print(example["tokens"])

['ใส่', 'กระดาษ', 'จะ', 'ทำให้', 'จำได้', 'ง่าย', ' ', ' ', 'เพราะ', ' ', 'ทำ', 'ใน', 'คอม', ' ', 'มีระบบ', ' ', 'ใส่', 'คำสั่ง', 'ให้', 'เอง', ' ', 'ถึง', 'เวลา', 'สอบ', 'สมัครงาน', 'บาง', 'ที่', ' ', 'ให้', 'เขียน', 'ลง', 'กระดาษ', ' ', 'ก็', 'จะ', 'เขียน', 'ไม่', 'ออก', ' ']


In [11]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['<s>', '▁', 'ใส่', '▁', 'กระดาษ', '▁จะ', '▁ทําให้', '▁', 'จําได้', '▁', 'ง่าย', '▁', '▁', '▁เพราะ', '▁', '▁ทํา', '▁', 'ใน', '▁', 'คอม', '▁', '▁', 'มีระบบ', '▁', '▁', 'ใส่', '▁', 'คําสั่ง', '▁', 'ให้', '▁', 'เอง', '▁', '▁', 'ถึง', '▁เวลา', '▁', 'สอบ', '▁', 'สมัครงาน', '▁บาง', '▁', 'ที่', '▁', '▁', 'ให้', '▁', 'เขียน', '▁', 'ลง', '▁', 'กระดาษ', '▁', '▁ก็', '▁จะ', '▁', 'เขียน', '▁ไม่', '▁', 'ออก', '▁', '</s>']


In [12]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

62 62


In [13]:
label_all_tokens = True

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenize_and_align_labels(datasets['train'][:1])

{'input_ids': [[5, 10, 10539, 10, 15486, 10, 6850, 10, 4738, 13764, 10, 1599, 10, 499, 10, 711, 10, 1599, 10, 662, 10, 711, 1739, 222, 6989, 10, 111, 10, 158, 10, 6]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, -100]]}

In [15]:
tokenized_datasets = datasets.map(tokenize_and_align_labels, batched=True)
tokenized_datasets

100%|██████████| 1/1 [00:00<00:00,  9.71ba/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 552
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 138
    })
})

## Fine-tuning the model

In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased were not used when initializing CamembertForTokenClassification: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing CamembertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing CamembertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier

In [24]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [25]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [26]:
from datasets import load_metric
metric = load_metric("seqeval")

In [27]:
labels = [label_list[i] for i in example[f"{task}_tags"]]
metric.compute(predictions=[labels], references=[labels])

{'c': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'p': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [28]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [30]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 552
  Num Epochs = 10
  Instantaneous batch size per device = 12
  Total train batch size (w. parallel, distributed & accumulation) = 12
  Gradient Accumulation steps = 1
  Total optimization steps = 460
 10%|█         | 46/460 [00:29<04:23,  1.57it/s]The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
    

{'eval_loss': 1.0115090608596802, 'eval_precision': 0.013196480938416423, 'eval_recall': 0.04275534441805225, 'eval_f1': 0.020168067226890758, 'eval_accuracy': 0.5455089223407437, 'eval_runtime': 2.473, 'eval_samples_per_second': 55.802, 'eval_steps_per_second': 4.852, 'epoch': 1.0}


 20%|██        | 92/460 [01:03<04:12,  1.46it/s]The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
                                                
 20%|██        | 92/460 [01:05<04:12,  1.46it/s]

{'eval_loss': 0.9268268346786499, 'eval_precision': 0.05484818805093046, 'eval_recall': 0.1330166270783848, 'eval_f1': 0.07766990291262137, 'eval_accuracy': 0.594357491775496, 'eval_runtime': 2.774, 'eval_samples_per_second': 49.748, 'eval_steps_per_second': 4.326, 'epoch': 2.0}


 30%|███       | 138/460 [01:37<03:39,  1.47it/s]The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
                                                 
 30%|███       | 138/460 [01:40<03:39,  1.47it/s]

...

 90%|█████████ | 414/460 [05:08<00:32,  1.43it/s]The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
                                                 
 90%|█████████ | 414/460 [05:11<00:32,  1.43it/s]

{'eval_loss': 1.051697015762329, 'eval_precision': 0.25796178343949044, 'eval_recall': 0.3847980997624703, 'eval_f1': 0.3088655862726406, 'eval_accuracy': 0.6618482703618782, 'eval_runtime': 3.045, 'eval_samples_per_second': 45.32, 'eval_steps_per_second': 3.941, 'epoch': 9.0}


100%|██████████| 460/460 [05:44<00:00,  1.42it/s]The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
                                                 
100%|██████████| 460/460 [05:47<00:00,  1.42it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 460/460 [05:47<00:00,  1.32it/s]

{'eval_loss': 1.0708237886428833, 'eval_precision': 0.25241157556270094, 'eval_recall': 0.37292161520190026, 'eval_f1': 0.30105465004793863, 'eval_accuracy': 0.6625461070680889, 'eval_runtime': 3.047, 'eval_samples_per_second': 45.29, 'eval_steps_per_second': 3.938, 'epoch': 10.0}
{'train_runtime': 347.26, 'train_samples_per_second': 15.896, 'train_steps_per_second': 1.325, 'train_loss': 0.5693019701086957, 'epoch': 10.0}


TrainOutput(global_step=460, training_loss=0.5693019701086957, metrics={'train_runtime': 347.26, 'train_samples_per_second': 15.896, 'train_steps_per_second': 1.325, 'train_loss': 0.5693019701086957, 'epoch': 10.0})

In [31]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 138
  Batch size = 12
100%|██████████| 12/12 [00:02<00:00,  4.67it/s]


{'eval_loss': 1.0708237886428833,
 'eval_precision': 0.25241157556270094,
 'eval_recall': 0.37292161520190026,
 'eval_f1': 0.30105465004793863,
 'eval_accuracy': 0.6625461070680889,
 'eval_runtime': 2.9017,
 'eval_samples_per_second': 47.559,
 'eval_steps_per_second': 4.136,
 'epoch': 10.0}

In [73]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set  don't have a corresponding argument in `CamembertForTokenClassification.forward` and have been ignored: pos_tags, ner_tags, tokens, id. If pos_tags, ner_tags, tokens, id are not expected by `CamembertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 138
  Batch size = 12
25it [07:17,  2.85s/it]

{'c': {'precision': 0.30618892508143325,
  'recall': 0.3983050847457627,
  'f1': 0.3462246777163904,
  'number': 236},
 'p': {'precision': 0.2,
  'recall': 0.34054054054054056,
  'f1': 0.25200000000000006,
  'number': 185},
 'overall_precision': 0.25241157556270094,
 'overall_recall': 0.37292161520190026,
 'overall_f1': 0.30105465004793863,
 'overall_accuracy': 0.6625461070680889}

In [75]:
idx = 20
show_text = tokenizer(tokenized_datasets["test"][idx]['tokens'], is_split_into_words=True)
print(tokenizer.convert_ids_to_tokens(show_text['input_ids']))

['<s>', '▁สําหรับ', '▁เรา', '▁', 'จบ', '▁', 'โท', '▁', '▁', 'ที่', '▁', '▁', 'ออสเตรเลีย', '▁', '▁และ', '▁', 'นิวซีแลนด์', '▁', '▁เรา', '▁', 'ว่า', '▁', 'คุ้ม', '▁', 'ยิ่งกว่า', '▁', 'คุ้ม', '▁', '▁', 'เปลี่ยน', '▁ชีวิต', '▁', '▁', 'มุมมอง', '▁', '▁', 'ทัศนคติ', '▁', '▁', 'คอนเน', 'ค', '▁', 'ชั่น', '▁', '▁งาน', '▁', '▁', 'เงิน', '▁แน่นอน', '▁', 'ว่า', '▁', 'ใช้เวลา', '▁', 'ค่ะ', '▁', '▁แต่', '▁ถ้า', '▁', 'มีโอกาส', '▁', 'ไป', '▁', 'เถอะ', '▁', 'ค่ะ', '▁', '▁คน', '▁มี', '▁', 'ความสามารถ', '▁', 'มักจะ', '▁มี', '▁', 'หนทาง', '▁', 'ต่อไป', '▁', 'เรื่อยๆ', '</s>']


In [76]:
print('pred:', true_predictions[idx])
print('true:', true_labels[idx])

pred: ['B-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'B-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p']
true: ['B-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'O', 'B-p', 'B-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'I-c', 'O

## Predict Text

In [49]:
from pythainlp.tokenize import word_tokenize
text_list = word_tokenize('ฉันชอบหมาเพราะมันน่ารักมากๆ เลย')
print(text_list)

['ฉัน', 'ชอบ', 'หมา', 'เพราะ', 'มัน', 'น่ารัก', 'มาก', 'ๆ', ' ', 'เลย']


In [50]:
_input_token = tokenizer(text_list, is_split_into_words=True)
_word_token = tokenizer.convert_ids_to_tokens(_input_token["input_ids"])
print(_input_token)
print(_word_token)

{'input_ids': [5, 264, 1879, 10, 1022, 474, 661, 5840, 10, 82, 10, 34, 10, 10, 48, 6], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
['<s>', '▁ฉัน', '▁ชอบ', '▁', 'หมา', '▁เพราะ', '▁มัน', '▁น่ารัก', '▁', 'มาก', '▁', 'ๆ', '▁', '▁', 'เลย', '</s>']


In [51]:
pred = trainer.predict([_input_token])[0]
pred = np.argmax(pred, axis=2)
pred

***** Running Prediction *****
  Num examples = 1
  Batch size = 12
14it [04:29, 63.92s/it]

array([[4, 1, 2, 2, 2, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]], dtype=int64)

In [52]:
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(pred, labels)
][0]
print(true_predictions)

['B-c', 'I-c', 'I-c', 'I-c', 'B-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p', 'I-p']


In [53]:
print(len(true_predictions))
print(len(_word_token))

15
16


In [54]:
for w, l in zip(_word_token[1:-1], true_predictions):
  print((w, l))

('▁ฉัน', 'B-c')
('▁ชอบ', 'I-c')
('▁', 'I-c')
('หมา', 'I-c')
('▁เพราะ', 'B-p')
('▁มัน', 'I-p')
('▁น่ารัก', 'I-p')
('▁', 'I-p')
('มาก', 'I-p')
('▁', 'I-p')
('ๆ', 'I-p')
('▁', 'I-p')
('▁', 'I-p')
('เลย', 'I-p')
